In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-midterm-p5/train.csv
/kaggle/input/2023-ml-midterm-p5/test.csv
/kaggle/input/2023-ml-midterm-p5/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
#데이터 불러오기
train = pd.read_csv("/kaggle/input/2023-ml-midterm-p5/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-midterm-p5/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-midterm-p5/submit_sample.csv")

In [4]:
#데이터 생김새 확인하기
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#데이터에 필요없는 칼럼 없애주기
train.drop(['PassengerId', 'Name', 'Ticket'],axis=1,inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket'],axis=1,inplace=True)

In [6]:
#결측치 및 type확인하기
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [7]:
#결측치가 너무 많은 칼럼 드랍하기
train.drop(['Cabin'],axis=1,inplace=True)
test.drop(['Cabin'],axis=1,inplace=True)

In [8]:
#결측치 및 type확인하기
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
#train데이터를 독립변수와 종속변수로 나눠주기
x = train.drop(['Survived'],axis=1)
y = train[['Survived']]

In [10]:
#x와 test데이터 합친 data만들어주기
data = pd.concat([x,test],axis=0)

In [11]:
from sklearn.preprocessing import LabelEncoder
#sex를 수치형 데이터로 바꿔주기
le_sex = LabelEncoder()
le_sex.fit(data['Sex'])
x['Sex']=le_sex.transform(x['Sex'])
test['Sex'] =le_sex.transform(test['Sex'])

#Embarked를 수치형 데이터로 바꿔주기
le_Embarked = LabelEncoder()
le_Embarked.fit(data['Embarked'])
x['Embarked']=le_Embarked.transform(x['Embarked'])
test['Embarked'] =le_Embarked.transform(test['Embarked'])

In [12]:
# 결측치 채우기
from sklearn.impute import SimpleImputer
si = SimpleImputer()
si.fit(data[['Age']])
x[['Age']] = si.transform(x[['Age']])
test[['Age']] = si.transform(test[['Age']])

In [13]:
#scaling 해주기
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_st = ss.fit_transform(x)
test_st = ss.transform(test)

In [14]:
#모델 학습 및 정확도 확인
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model=LogisticRegression(max_iter=5000)
model.fit(x_st,y['Survived'])
print(accuracy_score(y['Survived'],model.predict(x_st)))

0.8013468013468014


In [15]:
#submit형태확인
submit.head()

,PassengerId,Survived
0,892,-1
1,893,-1
2,894,-1
3,895,-1
4,896,-1


In [16]:
# 제출
submit['Survived']=model.predict(test_st)
submit.to_csv("submit.csv",mode='w',header=True,index=False)